Prosseguimos o desenvolvimento do trabalho elaborando um estudo de simulações a partir do conjunto de imagens Fashion-MNIST. Para cada classe de roupa (10 classes) atribuímos um valor específico para $\theta(x)$. Deste modo, esperamos que o modelo seja capaz de diferenciar os tipos de imagens e estimar o parâmetro associado a elas corretamente.

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import random
from time import time

from scipy.special import comb, loggamma, lambertw

import mps
import pwexp

E0000 00:00:1741151782.295236 1835015 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741151782.298727 1835015 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# Valor máximo do suporte da distribuição
B = 10001

# ---------------------------- Poisson (Pocr) ----------------------------
# Versões sem o tensorflow
log_a_poisson = lambda m : -loggamma(m+1)
log_phi_poisson = lambda theta : np.log(theta)
C_poisson = lambda theta : np.exp(theta)
C_inv_poisson = lambda u : np.log(u)
# Versões para o tensorflow
log_a_poisson_tf = lambda m : -tf.math.lgamma(m+1)
log_phi_poisson_tf = lambda theta : tf.math.log(theta)
C_poisson_tf = lambda theta : tf.math.exp(theta)
C_inv_poisson_tf = lambda u : tf.math.log(u)
sup_poisson = np.arange(0, B, 1).astype(np.float64)

theta_min_poisson = None
theta_max_poisson = None
def E_poisson(theta):
    return theta
# Variance is always equal to the mean
def Var_poisson(theta):
    return theta

# ---------------------------- Logarithmic (Locr) ----------------------------
# Versões sem o tensorflow
log_a_log = lambda m : -np.log(m+1)
log_phi_log = lambda theta : np.log(theta)
C_log = lambda theta : -np.log(1-theta)/theta
C_inv_log = lambda u : 1 + np.real(lambertw(-u*np.exp(-u))) / u
# Versões para o tensorflow
log_a_log_tf = lambda m : -tf.math.log(m+1)
log_phi_log_tf = lambda theta : tf.math.log(theta)
C_log_tf = lambda theta : -tf.math.log(1-theta)/theta
C_inv_log_tf = lambda u : 1 + tfp.math.lambertw(-u*tf.math.exp(-u)) / u
sup_log = np.arange(0, B, 1).astype(np.float64)

theta_min_log = 0
theta_max_log = 1
def E_log(theta):
    return -theta / (np.log(1-theta)*(1-theta)) - 1
# Overdispersion: in this case, variance is always greater than mean
def Var_log(theta):
    return -theta*(theta + np.log(1-theta)) / ((1-theta)**2*(np.log(1-theta))**2)

# ---------------------------- Binomial Negativa (+Geométrica) (NBcr + Gecr) ----------------------------
# Versões sem o tensorflow
def log_a_nb(q):
    return lambda m : loggamma(m+q) - loggamma(m+1) - loggamma(q)
def log_phi_nb(q):
    return lambda theta : np.log(1-theta)
def C_nb(q):
    return lambda theta : theta**(-q)
def C_inv_nb(q):
    return lambda u : u**(-1/q)
# Versões para o tensorflow
def log_a_nb_tf(q):
    return lambda m : tf.math.lgamma(m+q) - tf.math.lgamma(m+1) - tf.math.lgamma(q)
def log_phi_nb_tf(q):
    return lambda theta : tf.math.log(1-theta)
def C_nb_tf(q):
    return lambda theta : theta**(-q)
def C_inv_nb_tf(q):
    return lambda u : u**(-1/q)
sup_nb = np.arange(0, B, 1).astype(np.float64)

theta_min_nb = 0
theta_max_nb = 1
def E_nb(q, theta):
    return q*(1-theta)/theta
# Overdispersion: in this case, variance is always greater than mean
def Var_nb(q, theta):
    return q*(1-theta)/theta**2

# ---------------------------- Binomial (+Bernoulli) (Bincr + Bercr) ----------------------------
# Versões sem o tensorflow
def log_a_bin(q):
    return lambda m : loggamma(q+1) - loggamma(m+1) - loggamma(q-m+1)
def log_phi_bin(q):
    return lambda theta : np.log(theta) - np.log(1-theta)
def C_bin(q):
    return lambda theta : (1-theta)**(-q)
def C_inv_bin(q):
    return lambda u : 1 - u**(-1/q)
# Versões para o tensorflow
def log_a_bin_tf(q):
    return lambda m : tf.math.lgamma(q+1) - tf.math.lgamma(m+1) - tf.math.lgamma(q-m+1)
def log_phi_bin_tf(q):
    return lambda theta : tf.math.log(theta) - tf.math.log(1-theta)
def C_bin_tf(q):
    return lambda theta : (1-theta)**(-q)
def C_inv_bin_tf(q):
    return lambda u : 1 - u**(-1/q)
def sup_bin(q):
    return np.arange(0, q+1, 1).astype(np.float64)

theta_min_bin = 0
theta_max_bin = 1
def E_bin(q, theta):
    return q*theta
# Underdispersion: in this case, variance is always lesser than mean
def Var_bin(q, theta):
    return q*theta*(1-theta)

# ---------------------------- Borel (Bocr) ----------------------------
# Versões sem o tensorflow
log_a_borel = lambda m : (m-1)*np.log(m+1) - loggamma(m+1)
log_phi_borel = lambda theta : np.log(theta) - theta
C_borel = lambda theta : np.exp(theta)
C_inv_borel = lambda u : np.log(u)
# Versões para o tensorflow
log_a_borel_tf = lambda m : (m-1)*tf.math.log(m+1) - tf.math.lgamma(m+1)
log_phi_borel_tf = lambda theta : tf.math.log(theta) - theta
C_borel_tf = lambda theta : tf.math.exp(theta)
C_inv_borel_tf = lambda u : tf.math.log(u)
sup_borel = np.arange(0, B, 1).astype(np.float64)

theta_min_borel = 0
theta_max_borel = 1
def E_borel(theta):
    return theta/(1-theta)
# Overdispersion: in this case, variance is always greater than mean
def Var_borel(theta):
    mu = theta/(1-theta)
    return mu*(1+mu)**2

# ---------------------------- Restricted Generalized Poisson (RGPcr) ----------------------------
# Versões sem o tensorflow
def log_a_rgp(q):
    return lambda m : (m-1)*np.log(1+q*m) - loggamma(m+1)
def log_phi_rgp(q):
    return lambda theta : np.log(theta) - q*theta
def C_rgp(q):
    return lambda theta : np.exp(theta)
def C_inv_rgp(q):
    return lambda u : np.log(u)
# Versões para o tensorflow
def log_a_rgp_tf(q):
    return lambda m : (m-1)*tf.math.log(1+q*m) - tf.math.lgamma(m+1)
def log_phi_rgp_tf(q):
    return lambda theta : tf.math.log(theta) - q*theta
def C_rgp_tf(q):
    return lambda theta : tf.math.exp(theta)
def C_inv_rgp_tf(q):
    return lambda u : tf.math.log(u)
def sup_rgp(q):
    if(q > 0):
        return np.arange(0, B, 1).astype(np.float64)
    else:
        if(q < -1):
            raise Exception("q value can't be less than -1")
        max_sup_candidates = np.arange(1, 101)
        max_sup = max_sup_candidates[(1 + max_sup_candidates*q) > 0][-1]
        return np.arange(max_sup+1)

theta_min_rgp = 0
# The RGP theta parameter must be lesser than q, otherwise its probabilities do not sum to one
def theta_max_rgp(q):
    return np.abs(1/q)
def E_rgp(q, theta):
    return theta/(1-q*theta)
# Overdispersion: in this case, variance is always greater than mean
def Var_rgp(q, theta):
    return theta / (1-q*theta)**3

# ---------------------------- Haight (Catalan) (Cacr) - Geeta(q = 2) ----------------------------
# Versões sem o tensorflow
log_a_haight = lambda m : loggamma(2*m+2) - loggamma(m+2) - loggamma(m+1) - np.log(2*m+1)
log_phi_haight = lambda theta : np.log(theta) + np.log(1-theta)
C_haight = lambda theta : 1/(1-theta)
C_inv_haight = lambda u : 1 - 1/u
# Versões para o tensorflow
log_a_haight_tf = lambda m : tf.math.lgamma(2*m+2) - tf.math.lgamma(m+2) - tf.math.lgamma(m+1) - tf.math.log(2*m+1)
log_phi_haight_tf = lambda theta : tf.math.log(theta) + tf.math.log(1-theta)
C_haight_tf = lambda theta : 1/(1-theta)
C_inv_haight_tf = lambda u : 1 - 1/u
sup_haight = np.arange(0, B, 1).astype(np.float64)

theta_min_haight = 0
theta_max_haight = 0.5
def E_haight(theta):
    p = theta
    s = 1-p
    return s/(s-p) - 1
# Overdispersion: in this case, variance is always greater than mean
def Var_haight(theta):
    p = theta
    s = 1-p
    return p*s/(s-p)**2 + 2*p**2*s/(s-p)**3

# ---------------------------- Geeta (Gecr) ----------------------------
# Versões sem o tensorflow
def log_a_geeta(q):
    return lambda m : loggamma(q*m+q) - loggamma(m+2) - loggamma((q-1)*m+q-1) - np.log(q*m+q-1)
def log_phi_geeta(q):
    return lambda theta : np.log(theta) + (q-1)*np.log(1-theta)
def C_geeta(q):
    return lambda theta : (1-theta)**(1-q)
def C_inv_geeta(q):
    return lambda u : 1 - u**(1/(1-q))
# Versões para o tensorflow
def log_a_geeta_tf(q):
    return lambda m : tf.math.lgamma(q*m+q) - tf.math.lgamma(m+2) - tf.math.lgamma(q*m+q) - tf.math.log(q*m+q-1)
def log_phi_geeta_tf(q):
    return lambda theta : tf.math.log(theta) + (q-1)*tf.math.log(1-theta)
def C_geeta_tf(q):
    return lambda theta : 1/(1-theta)
def C_inv_geeta_tf(q):
    return lambda u : 1 - u**(1/(1-q))
sup_geeta = np.arange(0, B, 1).astype(np.float64)

theta_min_geeta = 0
theta_max_geeta = lambda q : 1/q
def E_geeta(q, theta):
    return (1-theta)/(1-q*theta) - 1
def Var_geeta(q, theta):
    mu = (1-theta)/(1-q*theta)
    return mu*(mu-1)*(mu*q-1)/(q-1)

In [3]:
# ---------------------------- Binomial Negativa (Mean-Variance parametrization) (MVNBcr) ----------------------------
# Versões sem o tensorflow
def log_a_mvnb(q):
    return lambda m : loggamma(1/q+m) - loggamma(1/q) - loggamma(m+1)
def log_phi_mvnb(q):
    return lambda theta : np.log(q*theta) - np.log(1+q*theta)
def C_mvnb(q):
    return lambda theta : (1 + q*theta)**(1/q)
def C_inv_mvnb(q):
    return lambda u : (u**q - 1)/q
# Versões para o tensorflow
def log_a_mvnb_tf(q):
    return lambda m : tf.math.lgamma(1/q+m) - tf.math.lgamma(1/q) - tf.math.lgamma(m+1)
def log_phi_mvnb_tf(q):
    return lambda theta : tf.math.log(q*theta) - tf.math.log(1+q*theta)
def C_mvnb_tf(q):
    return lambda theta : (1 + q*theta)**(1/q)
def C_inv_mvnb_tf(q):
    return lambda u : (u**q - 1) / q
sup_nb = np.arange(0, B, 1).astype(np.float64)

theta_min_nb = None
theta_max_nb = None
def E_mvnb(q, theta):
    return theta
# Overdispersion: in this case, variance is always greater than mean
def Var_mvnb(q, theta):
    return (1+q*theta)*theta

In [4]:
def sim_dist(log_a, log_phi, sup, theta, n):
    x = mps.rvs(log_a, log_phi, theta, sup, size = n)
    x_mean = np.mean(x)
    x_var = np.var(x)
    print("    Sample mean: {:.6f}".format(x_mean))
    print("    Sample var: {:.6f}".format(x_var))
    
def sim_dist_thetas(log_a, log_phi, sup, thetas, n, E_func, Var_func, q = None):
    for theta in thetas:
        print("--- theta = {:.4f}".format(theta))
        if(q is None):
            print("    Theoretical mean: {:.6f}".format(E_func(theta)))
            print("    Theoretical variance: {:.6f}".format(Var_func(theta)))
        else:
            print("    Theoretical mean: {:.6f}".format(E_func(q, theta)))
            print("    Theoretical variance: {:.6f}".format(Var_func(q, theta)))
        sim_dist(log_a, log_phi, sup, theta, n)

In [5]:
n = 500000

## Poisson distribution

In [6]:
print("Poisson distribution")
sim_dist_thetas(log_a_poisson, log_phi_poisson, sup_poisson, [0.5, 1, 2, 5, 10, 20], n, E_poisson, Var_poisson, q = None)

Poisson distribution
--- theta = 0.5000
    Theoretical mean: 0.500000
    Theoretical variance: 0.500000
    Sample mean: 0.501422
    Sample var: 0.501098
--- theta = 1.0000
    Theoretical mean: 1.000000
    Theoretical variance: 1.000000
    Sample mean: 1.001272
    Sample var: 1.001574
--- theta = 2.0000
    Theoretical mean: 2.000000
    Theoretical variance: 2.000000
    Sample mean: 2.001414
    Sample var: 1.999164
--- theta = 5.0000
    Theoretical mean: 5.000000
    Theoretical variance: 5.000000
    Sample mean: 4.997450
    Sample var: 5.007951
--- theta = 10.0000
    Theoretical mean: 10.000000
    Theoretical variance: 10.000000
    Sample mean: 10.016558
    Sample var: 10.012800
--- theta = 20.0000
    Theoretical mean: 20.000000
    Theoretical variance: 20.000000
    Sample mean: 20.000708
    Sample var: 19.946779


## Logarithmic distribution

In [7]:
print("Logarithmic distribution")
sim_dist_thetas(log_a_log, log_phi_log, sup_log, [0.1, 0.25, 0.5, 0.75, 0.9], n, E_log, Var_log, q = None)

Logarithmic distribution
--- theta = 0.1000
    Theoretical mean: 0.054580
    Theoretical variance: 0.059616
    Sample mean: 0.054404
    Sample var: 0.059332
--- theta = 0.2500
    Theoretical mean: 0.158686
    Theoretical variance: 0.202361
    Sample mean: 0.157596
    Sample var: 0.201144
--- theta = 0.5000
    Theoretical mean: 0.442695
    Theoretical variance: 0.804021
    Sample mean: 0.440930
    Sample var: 0.800999
--- theta = 0.7500
    Theoretical mean: 1.164043
    Theoretical variance: 3.973090
    Sample mean: 1.163546
    Sample var: 3.960883
--- theta = 0.9000
    Theoretical mean: 2.908650
    Theoretical variance: 23.808956
    Sample mean: 2.904266
    Sample var: 23.828269


## NB(q = 1) distribution (Geometric distribution)

In [8]:
print("Geometric distribution")
sim_dist_thetas(log_a_nb(1), log_phi_nb(1), sup_nb, [0.1, 0.25, 0.5, 0.75, 0.9], n, E_nb, Var_nb, q = 1)

Geometric distribution
--- theta = 0.1000
    Theoretical mean: 9.000000
    Theoretical variance: 90.000000
    Sample mean: 8.996290
    Sample var: 89.730168
--- theta = 0.2500
    Theoretical mean: 3.000000
    Theoretical variance: 12.000000
    Sample mean: 3.000770
    Sample var: 11.995049
--- theta = 0.5000
    Theoretical mean: 1.000000
    Theoretical variance: 2.000000
    Sample mean: 1.000884
    Sample var: 1.998895
--- theta = 0.7500
    Theoretical mean: 0.333333
    Theoretical variance: 0.444444
    Sample mean: 0.333212
    Sample var: 0.443202
--- theta = 0.9000
    Theoretical mean: 0.111111
    Theoretical variance: 0.123457
    Sample mean: 0.110830
    Sample var: 0.122859


In [9]:
# theta = (1-p)/p, p being the probability for the geometric
print("Mean-Variance Negative Binomial distribution (q = 1) = Geometric")
sim_dist_thetas(log_a_mvnb(1), log_phi_mvnb(1), sup_nb, [9, 3, 1, 1/3, 1/9], n, E_mvnb, Var_mvnb, q = 1)

Mean-Variance Negative Binomial distribution (q = 1) = Geometric
--- theta = 9.0000
    Theoretical mean: 9.000000
    Theoretical variance: 90.000000
    Sample mean: 9.007718
    Sample var: 89.753946
--- theta = 3.0000
    Theoretical mean: 3.000000
    Theoretical variance: 12.000000
    Sample mean: 2.998210
    Sample var: 12.022455
--- theta = 1.0000
    Theoretical mean: 1.000000
    Theoretical variance: 2.000000
    Sample mean: 0.998928
    Sample var: 1.999647
--- theta = 0.3333
    Theoretical mean: 0.333333
    Theoretical variance: 0.444444
    Sample mean: 0.334206
    Sample var: 0.447928
--- theta = 0.1111
    Theoretical mean: 0.111111
    Theoretical variance: 0.123457
    Sample mean: 0.110706
    Sample var: 0.122950


## NB(q = 1/2) distribution

In [24]:
print("NB(1/2) distribution")
sim_dist_thetas(log_a_mvnb(1/2), log_phi_mvnb(1/2), sup_nb, [9, 3, 1, 1/3, 1/9], n, E_mvnb, Var_mvnb, q = 1/2)

NB(1/2) distribution
--- theta = 9.0000
    Theoretical mean: 9.000000
    Theoretical variance: 49.500000
    Sample mean: 8.994464
    Sample var: 49.659565
--- theta = 3.0000
    Theoretical mean: 3.000000
    Theoretical variance: 7.500000
    Sample mean: 2.997172
    Sample var: 7.457820
--- theta = 1.0000
    Theoretical mean: 1.000000
    Theoretical variance: 1.500000
    Sample mean: 0.998154
    Sample var: 1.499919
--- theta = 0.3333
    Theoretical mean: 0.333333
    Theoretical variance: 0.388889
    Sample mean: 0.334566
    Sample var: 0.390540
--- theta = 0.1111
    Theoretical mean: 0.111111
    Theoretical variance: 0.117284
    Sample mean: 0.111742
    Sample var: 0.117956


In [21]:
print("Mean-Variance Negative Binomial distribution (q = 1/2)")
sim_dist_thetas(log_a_mvnb(1/2), log_phi_mvnb(1/2), sup_nb, [0.22, 0.67, 2, 6, 18], n, E_mvnb, Var_mvnb, q = 1/2)

Mean-Variance Negative Binomial distribution (q = 1/2)
--- theta = 0.2200
    Theoretical mean: 0.220000
    Theoretical variance: 0.244200
    Sample mean: 0.220470
    Sample var: 0.244495
--- theta = 0.6700
    Theoretical mean: 0.670000
    Theoretical variance: 0.894450
    Sample mean: 0.666574
    Sample var: 0.890081
--- theta = 2.0000
    Theoretical mean: 2.000000
    Theoretical variance: 4.000000
    Sample mean: 1.999666
    Sample var: 3.993358
--- theta = 6.0000
    Theoretical mean: 6.000000
    Theoretical variance: 24.000000
    Sample mean: 5.998744
    Sample var: 24.025942
--- theta = 18.0000
    Theoretical mean: 18.000000
    Theoretical variance: 180.000000
    Sample mean: 17.984758
    Sample var: 179.536046


## Bin(q = 1) distribution (Bernoulli)

In [12]:
print("Bernoulli distribution")
sim_dist_thetas(log_a_bin(1), log_phi_bin(1), sup_bin(1), [0.1, 0.25, 0.5, 0.75, 0.9], n, E_bin, Var_bin, q = 1)

Bernoulli distribution
--- theta = 0.1000
    Theoretical mean: 0.100000
    Theoretical variance: 0.090000
    Sample mean: 0.099960
    Sample var: 0.089968
--- theta = 0.2500
    Theoretical mean: 0.250000
    Theoretical variance: 0.187500
    Sample mean: 0.251098
    Sample var: 0.188048
--- theta = 0.5000
    Theoretical mean: 0.500000
    Theoretical variance: 0.250000
    Sample mean: 0.499570
    Sample var: 0.250000
--- theta = 0.7500
    Theoretical mean: 0.750000
    Theoretical variance: 0.187500
    Sample mean: 0.750306
    Sample var: 0.187347
--- theta = 0.9000
    Theoretical mean: 0.900000
    Theoretical variance: 0.090000
    Sample mean: 0.900234
    Sample var: 0.089813


## Bin(q = 20) distribution

In [13]:
print("Bernoulli distribution")
sim_dist_thetas(log_a_bin(20), log_phi_bin(20), sup_bin(20), [0.1, 0.25, 0.5, 0.75, 0.9], n, E_bin, Var_bin, q = 20)

Bernoulli distribution
--- theta = 0.1000
    Theoretical mean: 2.000000
    Theoretical variance: 1.800000
    Sample mean: 1.999282
    Sample var: 1.795061
--- theta = 0.2500
    Theoretical mean: 5.000000
    Theoretical variance: 3.750000
    Sample mean: 4.997318
    Sample var: 3.766319
--- theta = 0.5000
    Theoretical mean: 10.000000
    Theoretical variance: 5.000000
    Sample mean: 10.001310
    Sample var: 4.987396
--- theta = 0.7500
    Theoretical mean: 15.000000
    Theoretical variance: 3.750000
    Sample mean: 14.997236
    Sample var: 3.752888
--- theta = 0.9000
    Theoretical mean: 18.000000
    Theoretical variance: 1.800000
    Sample mean: 18.000860
    Sample var: 1.802119


## Borel distribution

In [14]:
print("Borel distribution")
sim_dist_thetas(log_a_borel, log_phi_borel, sup_borel, [0.1, 0.25, 0.5, 0.75, 0.9], n, E_borel, Var_borel, q = None)

Borel distribution
--- theta = 0.1000
    Theoretical mean: 0.111111
    Theoretical variance: 0.137174
    Sample mean: 0.110984
    Sample var: 0.136735
--- theta = 0.2500
    Theoretical mean: 0.333333
    Theoretical variance: 0.592593
    Sample mean: 0.331638
    Sample var: 0.587398
--- theta = 0.5000
    Theoretical mean: 1.000000
    Theoretical variance: 4.000000
    Sample mean: 1.001892
    Sample var: 4.026340
--- theta = 0.7500
    Theoretical mean: 3.000000
    Theoretical variance: 48.000000
    Sample mean: 2.987624
    Sample var: 47.704095
--- theta = 0.9000
    Theoretical mean: 9.000000
    Theoretical variance: 900.000000
    Sample mean: 9.033280
    Sample var: 902.303072


## RGP(q = 2) distribution

In [15]:
print("RGP distribution")
sim_dist_thetas(log_a_rgp(2), log_phi_rgp(2), sup_rgp(2), [0.1, 0.2, 0.3, 0.4, 0.49], n, E_rgp, Var_rgp, q = 2)

RGP distribution
--- theta = 0.1000
    Theoretical mean: 0.125000
    Theoretical variance: 0.195312
    Sample mean: 0.125644
    Sample var: 0.197622
--- theta = 0.2000
    Theoretical mean: 0.333333
    Theoretical variance: 0.925926
    Sample mean: 0.333822
    Sample var: 0.925933
--- theta = 0.3000
    Theoretical mean: 0.750000
    Theoretical variance: 4.687500
    Sample mean: 0.749890
    Sample var: 4.684027
--- theta = 0.4000
    Theoretical mean: 2.000000
    Theoretical variance: 50.000000
    Sample mean: 2.005448
    Sample var: 50.367850
--- theta = 0.4900
    Theoretical mean: 24.500000
    Theoretical variance: 61250.000000
    Sample mean: 22.920024
    Sample var: 42988.962488


## RGP(q = -1/10) distribution

In [16]:
print("RGP distribution")
sim_dist_thetas(log_a_rgp(-1/10), log_phi_rgp(-1/10), sup_rgp(-1/10), [0.5, 1, 2, 5, 9], n, E_rgp, Var_rgp, q = -1/10)

RGP distribution
--- theta = 0.5000
    Theoretical mean: 0.476190
    Theoretical variance: 0.431919
    Sample mean: 0.474686
    Sample var: 0.431011
--- theta = 1.0000
    Theoretical mean: 0.909091
    Theoretical variance: 0.751315
    Sample mean: 0.909804
    Sample var: 0.753949
--- theta = 2.0000
    Theoretical mean: 1.666667
    Theoretical variance: 1.157407
    Sample mean: 1.665562
    Sample var: 1.154617
--- theta = 5.0000
    Theoretical mean: 3.333333
    Theoretical variance: 1.481481
    Sample mean: 3.334328
    Sample var: 1.480541
--- theta = 9.0000
    Theoretical mean: 4.736842
    Theoretical variance: 1.312145
    Sample mean: 4.738720
    Sample var: 1.313857


## Haight distribution

In [17]:
print("Haight distribution")
sim_dist_thetas(log_a_haight, log_phi_haight, sup_haight, [0.1, 0.2, 0.3, 0.4, 0.49], n, E_haight, Var_haight, q = None)

Haight distribution
--- theta = 0.1000
    Theoretical mean: 0.125000
    Theoretical variance: 0.175781
    Sample mean: 0.126590
    Sample var: 0.179141
--- theta = 0.2000
    Theoretical mean: 0.333333
    Theoretical variance: 0.740741
    Sample mean: 0.333708
    Sample var: 0.742463
--- theta = 0.3000
    Theoretical mean: 0.750000
    Theoretical variance: 3.281250
    Sample mean: 0.748890
    Sample var: 3.265446
--- theta = 0.4000
    Theoretical mean: 2.000000
    Theoretical variance: 30.000000
    Sample mean: 2.009500
    Sample var: 30.605486
--- theta = 0.4900
    Theoretical mean: 24.500000
    Theoretical variance: 31237.500000
    Sample mean: 24.391162
    Sample var: 29994.882406


## Geeta(q = 3) distribution

In [18]:
print("Geeta distribution (q = 3)")
sim_dist_thetas(log_a_geeta(3), log_phi_geeta(3), sup_geeta, [0.05, 0.1, 0.2, 0.3, 0.33], n, E_geeta, Var_geeta, q = 3)

Geeta distribution (q = 3)
--- theta = 0.0500
    Theoretical mean: 0.117647
    Theoretical variance: 0.154692
    Sample mean: 0.117640
    Sample var: 0.154637
--- theta = 0.1000
    Theoretical mean: 0.285714
    Theoretical variance: 0.524781
    Sample mean: 0.285984
    Sample var: 0.528661
--- theta = 0.2000
    Theoretical mean: 1.000000
    Theoretical variance: 5.000000
    Sample mean: 1.004898
    Sample var: 5.086878
--- theta = 0.3000
    Theoretical mean: 6.000000
    Theoretical variance: 420.000000
    Sample mean: 6.013112
    Sample var: 423.933852
--- theta = 0.3300
    Theoretical mean: 66.000000
    Theoretical variance: 442200.000000
    Sample mean: 51.349232
    Sample var: 138823.525601
